# Final Poject

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

os.environ['OPENBLAS_NUM_THREADS'] = '1'
import random
import sys
from glob import glob
from random import shuffle

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

import h5py
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv3D, UpSampling3D, MaxPool3D, Concatenate, Input, Add, Subtract,ZeroPadding3D, BatchNormalization,ZeroPadding2D,UpSampling2D,Reshape,Conv2D
import pandas as pd
import skimage as sk
import torch
import torch.nn as nn
import torch.nn.functional as torchF
from scipy import ndarray
from sklearn.feature_extraction import image as sklearn_image
from torch.utils import data
from torchvision import transforms as T
from torchvision.transforms import functional as F
from torchvision.utils import save_image

import progressbar
import gc
import pickle
import math
# !pip install visualkeras
# import visualkeras



## Data Loading


In [ ]:
## Josh Loading Code
# Just for RAM reasons, we are only going to look at the first coil. This will reduce the dimensionality without 
# meaningfully reducing the information. 


def loadKspace(input_Folder, batch_size, batch_index):
  #Batch_size is the number of samples per training batch. 
  #Batch_Index is so that we can increment the batch number 

  #Create the paths to read the Input and Ground Truth Folders
  inputPath = input_Folder + "/Input/"
  GTPath  = input_Folder + "/GT/"

  #We have to make sure each input file has a corresponding GT file. Therefore we're going to load in all files name and compare them.
  inputNames = []
  GTNames = []

  for file in sorted(os.listdir(inputPath)):
    inputNames.append(file)
  for file in sorted(os.listdir(GTPath)):
    GTNames.append(file)

  #ValidInpute is a list of all the file names that appear in both the input and GT set. Must be ordered so that the batch code works correctly
  validInputs = set(inputNames).intersection(set(GTNames))
  validInputs = sorted(validInputs)
  print(str(len(validInputs)) + " Valid Sets")

  #Load in the inputs and GT. It's very inportant they load in the same order. Also create a progress bar
  widgets = [
           progressbar.Bar(),' (',
           progressbar.Percentage(), ') ',
          ]
  bar = progressbar.ProgressBar(max_value= batch_size, widgets=widgets).start()

  InputNps = []
  GTNps = []
  for i,fileName in enumerate(validInputs):
    if(i<batch_size*batch_index or i>=batch_size*(batch_index+1)):
      #ignore any samples outside of the current batch
      continue
    tempInput = np.load(inputPath+fileName) 
    tempGT = np.load(GTPath+fileName) 
    InputNps.append(tempInput)
    GTNps.append(tempGT)
    bar.update(i-batch_index*batch_size)
  
  return InputNps, GTNps





In [ ]:
def createMagnitude(kspaceArray):
  #From the KSpace Data we can create the magnitude data. Input is either inputs or GT
  widgets = [
           progressbar.Bar(),' (',
           progressbar.Percentage(), ') ',
          ]
  bar = progressbar.ProgressBar(max_value= len(kspaceArray), widgets=widgets).start()
  magArray = []
  for i,file in enumerate(kspaceArray):
    magnitude = np.fft.ifft2(file)
    magArray.append(magnitude)
    bar.update(i)
  
  return magArray
  




In [ ]:
#Combine the kspace and magnitude data.
inputsAll = []
GTAll = []


inputs, GT = loadKspace("/content/drive/My Drive/DLBI/Kspace/Val", batch_size = 25, batch_index = 0)
inputsMag = createMagnitude(inputs)
GTMag = createMagnitude(GT)

while(len(inputs)>0):
  inputsAll.append(np.stack([inputs[0],inputsMag[0]], axis=-1))
  del inputs[0], inputsMag[0] #Remove the variables from the name space so that they can be garbage collected
  
  GTAll.append(np.stack([GT[0],GTMag[0]], axis=-1))
  del GT[0], GTMag[0] #Remove the variables from the name space so that they can be garbage collected

  gc.collect() #Garbage Collection to free RAM


In [ ]:
print(len(inputsAll))
print(inputsAll[0].shape)
print(len(GTAll))
print(GTAll[0].shape)

## Preprocessing of Data
The current shape of the data is (n, 4, 18, 201, 402, 2). First are going to seprate the slices and add them to the batch number. Shape is now: (n\*201, 4, 18, 402, 2). Next we are going to rotate the dimensions to (n\*201, 2, 4, 402, 18). Then we are going to upsample to create better numbers for the convolution, creating (n\*201, 2, 4, 448, 32)


In [ ]:
def preprocessing(rawInput):

  finalInput = []

  while(len(rawInput)>0):
    #rotate array from (4, 18, 201, 402, 2) to (201, 2, 4, 402, 18)
    rawInputRotate = np.rot90(rawInput[0],1,(0,2))       #((201, 18, 4, 402, 2))
    rawInputRotate = np.rot90(rawInputRotate,1,(1,4))    #((201, 2, 4, 402, 18))
    # print("Rotated: Shape went from: " + str(rawInput[0].shape) + " to " + str(rawInputRotate.shape))
    del rawInput[0]
    gc.collect()

    #Histogram norm step [ADD HERE IF YOU THINK WE NEED IT]

    #Pad last two axis to make convolution easier
    padWidthOne = int((448 - rawInputRotate.shape[-2])/2)
    padTwo = int((32 - rawInputRotate.shape[-1])/2)
    rawInputPadded = np.pad(rawInputRotate, ((0,0),(0,0),(0,0),(padWidthOne,padWidthOne),(padTwo,padTwo)), 'constant')
    # print("Padded: Shape went from: " + str(rawInputRotate.shape) + " to " + str(rawInputPadded.shape))
    del rawInputRotate
    gc.collect()
    
    for i in range(201):
      finalInput.append(rawInputPadded[i])
    del rawInputPadded
    gc.collect()
    print(len(finalInput))
    
  finalInputNp = np.array([xi for xi in finalInput])
  del finalInput
  gc.collect()
  return finalInputNp


In [ ]:
processedInputsAll = preprocessing(inputsAll)
processedGTAll = preprocessing(GTAll)

In [ ]:
# ## Pickle Processed file 
processedDataTrain = [processedInputsAll, processedGTAll]
# with open('/content/drive/My Drive/DLBI/Kspace/processedData25_0Train.pickle', 'wb') as handle:
#   pickle.dump(processedDataTrain, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
processedDataVal = [processedInputsAll, processedGTAll]
# with open('/content/drive/My Drive/DLBI/Kspace/processedData25_0Val.pickle', 'wb') as handle:
#   pickle.dump(processedDataVal, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('/content/drive/My Drive/DLBI/Kspace/processedData25_0Train.pickle', 'rb') as handle:
    processedDataTrain = pickle.load(handle)
with open('/content/drive/My Drive/DLBI/Kspace/processedData25_0Val.pickle', 'rb') as handle:
    processedDataVal = pickle.load(handle)

In [ ]:
finalInputs = processedDataTrain[0]
finalGT = processedDataTrain[1]
finalInputsVal = processedDataVal[0]
finalGTVal = processedDataVal[1]

In [ ]:
print(finalInputs.shape)
print(finalGT.shape)

## Classifier 

In [ ]:
#Model 6
def reduceDim(inputReal, inputImg):
    downReal = Conv3D(1, (1, 1, 1), padding= "same", data_format="channels_first")(inputReal)
    downImg = Conv3D(1, (1, 1, 1), padding= "same", data_format="channels_first")(inputImg)
    downSampleReal = Subtract()([downReal,downImg])
    downSampleImg = Add()([downReal,downImg])
    downSampleReal = Reshape((downSampleReal.shape[2:]))(downSampleReal)
    downSampleImg = Reshape((downSampleImg.shape[2:]))(downSampleImg)
    
    return downSampleReal,downSampleImg

def finalDimExpan(inputReal,inputImg):

    upRealOne = Conv2D(4, (1, 1), padding= "same", data_format="channels_first")(inputReal)
    upImgOne = Conv2D(4, (1, 1), padding= "same", data_format="channels_first")(inputImg)
    upSampleRealOne = Subtract()([upRealOne,upImgOne])
    upSampleImgOne = Add()([upRealOne,upImgOne])

    upReal = Reshape(target_shape=(1,upSampleRealOne.shape[1],upSampleRealOne.shape[2],upSampleRealOne.shape[3] ))(upSampleRealOne)
    upImg = Reshape(target_shape=(1,upSampleImgOne.shape[1],upSampleImgOne.shape[2],upSampleImgOne.shape[3] ))(upSampleImgOne)


    upReal = Conv3D(2, (1, 1, 1), padding= "same",  data_format="channels_first")(upReal)
    upImg = Conv3D(2, (1, 1, 1), padding= "same", data_format="channels_first")(upImg)
    upSampleReal = Subtract()([upReal,upImg])
    upSampleImg = Add()([upReal,upImg])
    
    return upSampleReal,upSampleImg

def downConvolution(inputReal,inputImg, filter):

    #Convolutions
    OneReal = Conv2D(filter, 3, padding= "same", data_format="channels_first")(inputReal)
    OneImg = Conv2D(filter, 3, padding= "same", data_format="channels_first")(inputImg)
    CVLayerOneReal = Subtract()([OneReal,OneImg])
    CVLayerOneImg = Add()([OneReal,OneImg])
    
    TwoReal = Conv2D(filter, 3, padding= "same", data_format="channels_first")(CVLayerOneReal)
    TwoImg = Conv2D(filter, 3, padding= "same", activation = "relu", data_format="channels_first")(CVLayerOneImg)
    CVLayerTwoReal = Subtract()([TwoReal,TwoImg])
    CVLayerTwoImg = Add()([TwoReal,TwoImg])

    ThreeReal = Conv2D(filter, 3, padding= "same", data_format="channels_first")(CVLayerTwoReal)
    ThreeImg = Conv2D(filter, 3, padding= "same", data_format="channels_first")(CVLayerTwoImg)
    CVLayerThreeReal = Subtract()([ThreeReal,ThreeImg])
    CVLayerThreeImg = Add()([ThreeReal,ThreeImg])

    #Pooling
    CVLayerTwoRealPad = ZeroPadding2D((1,1), data_format="channels_first")(CVLayerThreeReal)
    CVLayerTwoImgPad = ZeroPadding2D((1,1), data_format="channels_first")(CVLayerThreeImg)
    poolOneReal = Conv2D(filter, 3, padding= "valid",strides = 2, data_format="channels_first")(CVLayerTwoRealPad)
    poolOneImg = Conv2D(filter, 3, padding= "valid",strides = 2, data_format="channels_first")(CVLayerTwoImgPad)
    poolReal = Subtract()([poolOneReal,poolOneImg])
    poolImg = Add()([poolOneReal,poolOneImg])

    poolReal = poolReal
    poolImg = poolImg


    return poolReal,poolImg, CVLayerThreeReal, CVLayerThreeImg


def upConvolution(inputReal,inputImg, carryReal,carryImg, filter):

    combinationReal = Concatenate(axis=0)([inputReal, carryReal])
    combinationImg = Concatenate(axis=0)([inputImg, carryImg])

    #Convolutions
    OneReal = Conv2D(filter, 3, padding= "same",  data_format="channels_first")(combinationReal)
    OneImg = Conv2D(filter, 3, padding= "same", data_format="channels_first")(combinationImg)
    CVLayerOneReal = Subtract()([OneReal,OneImg])
    CVLayerOneImg = Add()([OneReal,OneImg])
    
    TwoReal = Conv2D(filter, 3, padding= "same", data_format="channels_first")(CVLayerOneReal)
    TwoImg = Conv2D(filter, 3, padding= "same",  data_format="channels_first")(CVLayerOneImg)
    CVLayerTwoReal = Subtract()([TwoReal,TwoImg])
    CVLayerTwoImg = Add()([TwoReal,TwoImg])

    ThreeReal = Conv2D(filter/2, 3, padding= "same",data_format="channels_first")(CVLayerTwoReal)
    ThreeImg = Conv2D(filter/2, 3, padding= "same",  data_format="channels_first")(CVLayerTwoImg)
    CVLayerThreeReal = Subtract()([ThreeReal,ThreeImg])
    CVLayerThreeImg = Add()([ThreeReal,ThreeImg])
    
    
    upsampleReal = UpSampling2D(size = (2,2), data_format="channels_first")(CVLayerThreeReal)
    upsampleImg = UpSampling2D(size = (2,2), data_format="channels_first")(CVLayerThreeImg)

    return upsampleReal,upsampleImg
    
def finalUpconv(inputReal,inputImg, carryReal,carryImg, filter):

    combinationReal = Concatenate(axis=0)([inputReal, carryReal])
    combinationImg = Concatenate(axis=0)([inputImg, carryImg])

    OneReal = Conv2D(filter, 3, padding= "same",  data_format="channels_first")(combinationReal)
    OneImg = Conv2D(filter, 3, padding= "same", data_format="channels_first")(combinationImg)
    CVLayerOneReal = Subtract()([OneReal,OneImg])
    CVLayerOneImg = Add()([OneReal,OneImg])
    
    TwoReal = Conv2D(filter, 3, padding= "same", data_format="channels_first")(CVLayerOneReal)
    TwoImg = Conv2D(filter, 3, padding= "same",  data_format="channels_first")(CVLayerOneImg)
    CVLayerTwoReal = Subtract()([TwoReal,TwoImg])
    CVLayerTwoImg = Add()([TwoReal,TwoImg])
    
    ThreeReal = Conv2D(filter, 1, padding= "same",  data_format="channels_first")(CVLayerTwoReal)
    ThreeImg = Conv2D(filter, 1, padding= "same",  data_format="channels_first")(CVLayerTwoImg)
    CVLayerThreeReal = Subtract()([ThreeReal,ThreeImg])
    CVLayerThreeImg = Add()([ThreeReal,ThreeImg])

    return CVLayerThreeReal, CVLayerThreeImg

def bottomConv(inputReal,inputImg , filter):

    OneReal = Conv2D(filter, 3, padding= "same",  data_format="channels_first")(inputReal)
    OneImg = Conv2D(filter, 3, padding= "same", data_format="channels_first")(inputImg)
    CVLayerOneReal = Subtract()([OneReal,OneImg])
    CVLayerOneImg = Add()([OneReal,OneImg])
    
    TwoReal = Conv2D(filter/2, 3, padding= "same",  data_format="channels_first")(CVLayerOneReal)
    TwoImg = Conv2D(filter/2, 3, padding= "same", data_format="channels_first")(CVLayerOneImg)
    CVLayerTwoReal = Subtract()([TwoReal,TwoImg])
    CVLayerTwoImg = Add()([TwoReal,TwoImg])

    upsampleReal = UpSampling2D(size = (2,2), data_format="channels_first")(CVLayerTwoReal)
    upsampleImg = UpSampling2D(size = (2,2), data_format="channels_first")(CVLayerTwoImg)

    return upsampleReal, upsampleImg


def make_model():

    startingFilter = 128
    input_layerReal = Input(shape= [2,4,448,32], name = "Real") 
    input_layerImg = Input(shape= [2,4,448,32], name = "Img") 
    print("Shape of input: " + str(input_layerReal.shape))

    ##Upsample the coil layer
    firstUpSampleReal,firstUpSampleImg = reduceDim(input_layerReal,input_layerImg)
    print("Shape of first upsample: " + str(firstUpSampleReal.shape))

    ## Down convolutions
    downConvOneReal,downConvOneImg, carryOneReal,carryOneImg = downConvolution(firstUpSampleReal, firstUpSampleImg, startingFilter)
    print("Shape of first down conv: " + str(downConvOneReal.shape))
    downConvTwoReal,downConvTwoImg, carryTwoReal,carryTwoImg = downConvolution(downConvOneReal, downConvOneImg, startingFilter*2)
    print("Shape of second down conv: " + str(downConvTwoReal.shape))
    downConvThreeReal,downConvThreeImg, carryThreeReal,carryThreeImg = downConvolution(downConvTwoReal, downConvTwoImg, startingFilter*4)
    print("Shape of three down conv: " + str(downConvThreeReal.shape))
    downConvFourReal,downConvFourImg, carryFourReal,carryFourImg = downConvolution(downConvThreeReal,downConvThreeImg, startingFilter*8)
    print("Shape of four down conv: " + str(downConvFourReal.shape))
    finalConvReal,finalConvImg  = bottomConv( downConvFourReal,downConvFourImg, startingFilter*16)
    print("Shape of final down conv: " + str(finalConvReal.shape))

    #Up Conv
    upConvOneReal,upConvOneImg = upConvolution(finalConvReal,finalConvImg, carryFourReal,carryFourImg, startingFilter*8)
    print("Shape of first up conv: " + str(upConvOneReal.shape))
    upConvTwoReal,upConvTwoImg = upConvolution(upConvOneReal,upConvOneImg, carryThreeReal,carryThreeImg, startingFilter*4)
    print("Shape of second up conv: " + str(upConvTwoReal.shape))
    upConvThreeReal,upConvThreeImg = upConvolution(upConvTwoReal,upConvTwoImg, carryTwoReal,carryTwoImg, startingFilter*2)
    print("Shape of third up conv: " + str(upConvThreeReal.shape))
    upConvFourReal,upConvFourImg = finalUpconv(upConvThreeReal,upConvThreeImg, carryOneReal,carryOneImg, startingFilter)
    print("Shape of final up conv: " + str(upConvFourReal.shape))

    #Down sample coil layer
    lastDownSampleReal,lastDownSampleImg  = finalDimExpan(upConvFourReal,upConvFourImg)
    print("Final Shape: " + str(lastDownSampleReal.shape))


    return keras.models.Model(inputs=[input_layerReal,input_layerImg ], outputs=[lastDownSampleReal,lastDownSampleImg])

def CustomLoss(yReal, yPred,):

  lossReal = keras.losses.MeanAbsoluteError()(yReal[0],yPred[0] )

  finalLoss = lossReal

  return finalLoss

model = make_model()
opt = tf.keras.optimizers.Adam(0.000001)
model.compile(loss = CustomLoss, optimizer=opt)
model.save_weights("/content/drive/My Drive/DLBI/model4.h5")



In [ ]:
# model.summary()

In [ ]:
finalInputs.shape

In [ ]:
metaBatch = 201

In [ ]:
while(gc.collect()>0):
  continue

In [ ]:
model.load_weights('/content/drive/My Drive/DLBI/model4.h5')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0,  patience=50, min_delta = 0.0001)
mc = ModelCheckpoint('/content/drive/My Drive/DLBI/best_model4.h5', monitor='val_loss', mode='min', verbose=0, save_best_only=True)
history = model.fit(
    [finalInputs.real,finalInputs.imag],
    [finalGT.real,finalGT.imag],
    epochs=10000,
    verbose = 1,
    batch_size = 8,
    validation_data = ([finalInputsVal.real,finalInputsVal.imag],[finalGTVal.real,finalGTVal.imag]),
    callbacks=[es, mc]
)

In [ ]:
model.load_weights('/content/drive/My Drive/DLBI/best_model4.h5')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0,  patience=50, min_delta = 0.0001)
mc = ModelCheckpoint('/content/drive/My Drive/DLBI/best_model4B2.h5', monitor='val_loss', mode='min', verbose=0, save_best_only=True)
history2 = model.fit(
    [finalInputs[metaBatch:metaBatch*2].real,finalInputs[metaBatch:metaBatch*2].imag],
    [finalGT[metaBatch:metaBatch*2].real,finalGT[metaBatch:metaBatch*2].imag],
    epochs=10000,
    verbose = 1,
    batch_size = 4,
    validation_data = ([finalInputsVal[metaBatch:metaBatch*2].real,finalInputsVal[metaBatch:metaBatch*2].imag],[finalGTVal[metaBatch:metaBatch*2].real,finalGTVal[metaBatch:metaBatch*2].imag]),
    callbacks=[es, mc]
)

In [ ]:
model.load_weights('/content/drive/My Drive/DLBI/best_model4B2.h5')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0,  patience=50, min_delta = 0.0001)
mc = ModelCheckpoint('/content/drive/My Drive/DLBI/best_model4B3.h5', monitor='val_loss', mode='min', verbose=0, save_best_only=True)
history3 = model.fit(
    [finalInputs[metaBatch*2:metaBatch*3].real,finalInputs[metaBatch*2:metaBatch*3].imag],
    [finalGT[metaBatch*2:metaBatch*3].real,finalGT[metaBatch*2:metaBatch*3].imag],
    epochs=10000,
    verbose = 1,
    batch_size = 4,
    validation_data = ([finalInputsVal[metaBatch*2:metaBatch*3].real,finalInputsVal[metaBatch*2:metaBatch*3].imag],[finalGTVal[metaBatch*2:metaBatch*3].real,finalGTVal[metaBatch*2:metaBatch*3].imag]),
    callbacks=[es, mc]
)

In [ ]:
model.load_weights('/content/drive/My Drive/DLBI/best_model4B3.h5')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0,  patience=50, min_delta = 0.0001)
mc = ModelCheckpoint('/content/drive/My Drive/DLBI/best_model4B4.h5', monitor='val_loss', mode='min', verbose=0, save_best_only=True)
history4 = model.fit(
    [finalInputs[metaBatch*3:metaBatch*4].real,finalInputs[metaBatch*3:metaBatch*4].imag],
    [finalGT[metaBatch*3:metaBatch*4].real,finalGT[metaBatch*3:metaBatch*4].imag],
    epochs=10000,
    verbose = 1,
    batch_size = 4,
    validation_data = ([finalInputsVal[metaBatch*3:metaBatch*4].real,finalInputsVal[metaBatch*3:metaBatch*4].imag],[finalGTVal[metaBatch*3:metaBatch*4].real,finalGTVal[metaBatch*3:metaBatch*4].imag]),
    callbacks=[es, mc]
)

In [ ]:
model.load_weights('/content/drive/My Drive/DLBI/best_model4B4.h5')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0,  patience=50, min_delta = 0.0001)
mc = ModelCheckpoint('/content/drive/My Drive/DLBI/best_model4B5.h5', monitor='val_loss', mode='min', verbose=0, save_best_only=True)
history4 = model.fit(
    [finalInputs[metaBatch*4:metaBatch*5].real,finalInputs[metaBatch*4:metaBatch*5].imag],
    [finalGT[metaBatch*4:metaBatch*5].real,finalGT[metaBatch*4:metaBatch*5].imag],
    epochs=10000,
    verbose = 1,
    batch_size = 4,
    validation_data = ([finalInputsVal[metaBatch*4:metaBatch*5].real,finalInputsVal[metaBatch*4:metaBatch*5].imag],[finalGTVal[metaBatch*4:metaBatch*5].real,finalGTVal[metaBatch*4:metaBatch*5].imag]),
    callbacks=[es, mc]
)

## Output of the UNET

In [ ]:
model.load_weights("/content/drive/My Drive/DLBI/best_model4B4.h5")

In [ ]:
while(gc.collect()>0):
  continue 

In [ ]:
metaBatch = 1000
prediction = model.predict([finalInputs[:metaBatch].real,finalInputs[:metaBatch].imag], batch_size = 4,)

In [ ]:
GTForPrediction = [finalGT[:metaBatch].real, finalGT[:metaBatch].imag]

In [ ]:
# ## Pickle Processed file 
# with open('/content/drive/My Drive/DLBI/Kspace/prediction25_0B1.pickle', 'wb') as handle:
#   pickle.dump(prediction, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# ## Pickle Processed file 
# with open('/content/drive/My Drive/DLBI/Kspace/predGT25_0B1.pickle', 'wb') as handle:
#   pickle.dump(GTForPrediction, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('/content/drive/My Drive/DLBI/Kspace/prediction25_0B1.pickle', 'rb') as handle:
    prediction = pickle.load(handle)

In [ ]:
with open('/content/drive/My Drive/DLBI/Kspace/predGT25_0B1.pickle', 'rb') as handle:
    GTForPrediction = pickle.load(handle)

In [ ]:
print(prediction[0].shape)
print(prediction[1].shape)
print(GTForPrediction[0].shape)
print(GTForPrediction[1].shape)

In [ ]:
def reformImage(pred):
  imgReal = pred[0]
  imgImg = pred[1]

  output = np.zeros((imgReal.shape),dtype="complex128")
  output = 1j*imgImg; output += imgReal

  firstImage = output[:201, 0, 1, 23:425, 7:25]
  plt.imshow(np.log(abs(firstImage[:,:,8])), cmap='gray');
  


## GAN

In [ ]:
import numpy as np 
import pandas as pd 
import os
from __future__ import print_function
import time
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm_notebook as tqdm
print("Complete")

In [ ]:
# This belowing code is for real number setup

batch_size = 32
samples=990

data=GTForPrediction[0]
#data=prediction[0]
data=data[0:samples]
train_data=data        # The train data is ground truth data 
train_loader=torch.from_numpy(train_data)
imgs = next(iter(train_loader))

data2=prediction[0]
#data2=GTForPrediction[0]
data2=data2[0:samples]
noise_data=data2      # Noise data means the output data from UNet or the predicted data from the Unet
noise_loader=torch.from_numpy(noise_data) 
imgs2 = next(iter(noise_loader))

print("complete")

In [ ]:
# This belowing code is for imagniary number setup

batch_size = 32

data11=GTForPrediction[1]
data11=data11[0:samples]
train_data11=data11        # The train data is ground truth data 
train_loader11=torch.from_numpy(train_data11)
imgs = next(iter(train_loader11))

data22=prediction[1]
data22=data22[0:samples]
noise_data22=data22      # Noise data means the output data from UNet or the predicted data from the Unet
noise_loader22=torch.from_numpy(noise_data22) 
imgs2 = next(iter(noise_loader22))

print("complete")

In [ ]:
# Build the weights
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [ ]:
# Create Generator
class G(nn.Module):
    def __init__(self):
        # Used to inherit the torch.nn Module
        super(G, self).__init__()
        # Meta Module - consists of different layers of Modules
        self.main = nn.Sequential(
                nn.Conv2d(4, 512, 3, stride=1, padding='same', bias=False),
                nn.BatchNorm2d(512),
                #nn.ReLU(True),
                nn.Conv2d(512, 256, 3, stride=1, padding='same', bias=False),
                nn.BatchNorm2d(256),
                #nn.ReLU(True),
                nn.Conv2d(256, 128, 3, stride=1, padding='same', bias=False),
                nn.BatchNorm2d(128),
                #nn.ReLU(True),
                nn.Conv2d(128, 64, 3, stride=1, padding='same', bias=False),
                nn.BatchNorm2d(64),
                #nn.ReLU(True),
                nn.Conv2d(64, 4, 3, stride=1, padding='same', bias=False),    # stride 4 to 20. 448 to 4.  64 back to 64
                #nn.Tanh()
                )
    def forward(self, input):
        output = self.main(input)
        return output

# Creating the generator
netG = G()
netG.apply(weights_init)

In [ ]:
# Defining the discriminator
class D(nn.Module):
    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
                nn.Conv2d(4, 16, 2, stride=2, padding=1, bias=False),        # 16 back to 64
                nn.LeakyReLU(negative_slope=0.2, inplace=True),
                nn.Conv2d(16, 128, 2, stride=2, padding=1, bias=False),      # 16 back to 64
                nn.BatchNorm2d(128),
                nn.LeakyReLU(negative_slope=0.2, inplace=True),
                nn.Conv2d(128, 256, 2, stride=2, padding=1, bias=False),      
                nn.BatchNorm2d(256),                                          
                nn.LeakyReLU(negative_slope=0.2, inplace=True),
                nn.Conv2d(256, 512, 2, stride=2, padding=1, bias=False),      # second 256 back to to 512
                nn.BatchNorm2d(512),                                           # second 256 back to to 512
                nn.LeakyReLU(negative_slope=0.2, inplace=True),
                nn.Conv2d(512, 1, 2, stride=4, padding=0, bias=False),    # second 1024 back to to 1 #strude 1 to 2.  8 to 1
                nn.Sigmoid()
                )
        
    def forward(self, input):
        output = self.main(input)
        return output.view(-1)
        #return output
    
    
# Creating the discriminator
netD = D()
netD.apply(weights_init)

In [ ]:
# # To stored the final MRI images
# !mkdir MRIresults
# !ls

In [ ]:
EPOCH = 3 # play with the parameters
LR = 0.001
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=LR, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=LR, betas=(0.5, 0.999))

In [ ]:
# The belowing code is for real number training

import numpy
import tensorflow as tf
for epoch in range(EPOCH):
    for i,data1,data2 in zip(range(len(train_loader)),train_loader,noise_loader):
      # 1st Step: Updating the weights of the neural network of the discriminator
      netD.zero_grad()
    
      # Training the discriminator with a real image data of the dataset
      real1 = data1.float()
      input = Variable(real1)
      #target = Variable(torch.ones(input.size()[0]))
      #print(target.shape)
      target = Variable(torch.ones(14)) # 28 back to 14
      output = netD(input)
      #print("this is for output")
      #print(output.shape)
      errD_real = criterion(output, target)
      errD_real.backward(retain_graph=True) 

      # Training the generator with a fake image data
      data=data2.float()
      noise = data       # Change back back 
      #print("this is for noise type")
      #print(type(noise))
      #noise = Variable(torch.randn(input.size()[0], 4, 1, 1))    # cancel later
      noise = Variable(noise)
      #print("this is for noise shape")
      #print(noise.shape)

      fake2 = netG(noise)
      # print("fake 2 shape")
      # print(fake2.shape)
      #target = Variable(torch.zeros(noise.size()[0]))
      #print(target.shape)
      target = Variable(torch.zeros(14))
      output = netD(fake2.detach())
      #print(output.shape)
      errD_fake = criterion(output, target)
      fake2 = netG(noise)

      #target = Variable(torch.zeros(28672))
      #output = netD(fake.detach())
      #errD_fake = criterion(output, target)
      #errD_fake.backward(retain_graph=True)
        
      # Backpropagating the total error
      errD = errD_real + errD_fake
      errD.backward()
      optimizerD.step()
        
      # 2nd Step: Updating the weights of the neural network of the generator
      netG.zero_grad()
      #target = Variable(torch.ones(input.size()[0]))
      target = Variable(torch.ones(14))
      output = netD(fake2)
      errG = criterion(output, target)
      errG.backward()
      optimizerG.step()
        
      # 3rd Step: Printing the losses 
      print('[%d/%d][%d/%d] Loss_D: %.4f; Loss_G: %.4f' % (epoch+1, EPOCH, i+1, len(noise_loader), errD.item(), errG.item()))
        

In [ ]:
# The belowing code is for imaginary number training

import numpy
import tensorflow as tf
for epoch in range(EPOCH):
    for i,data1,data2 in zip(range(len(train_loader11)),train_loader11,noise_loader22):
      # 1st Step: Updating the weights of the neural network of the discriminator
      netD.zero_grad()
    
      # Training the discriminator with a real image of the dataset
      real11 = data1.float()
      input = Variable(real11)


      #target = Variable(torch.ones(input.size()[0]))
      target = Variable(torch.ones(14))
      output = netD(input)
      errD_real = criterion(output, target)
      errD_real.backward(retain_graph=True) 

      # Training the generator with a fake image
      data=data2.float()
      noise = data
      noise = Variable(noise)

      fake = netG(noise)
      target = Variable(torch.zeros(14))
      output = netD(fake.detach())
      errD_fake = criterion(output, target)
      fake22 = netG(noise)



      #target = Variable(torch.zeros(28672))
      #output = netD(fake.detach())
      #errD_fake = criterion(output, target)
      #errD_fake.backward(retain_graph=True)
        
      # Backpropagating the total error
      errD = errD_real + errD_fake
      errD.backward()
      optimizerD.step()
        
      # 2nd Step: Updating the weights of the neural network of the generator
      netG.zero_grad()
      #target = Variable(torch.ones(input.size()[0]))
      target = Variable(torch.ones(14))
      output = netD(fake22)
      errG = criterion(output, target)
      errG.backward()
      optimizerG.step()
        
      # 3rd Step: Printing the losses and saving the real images and the generated images of the minibatch every 100 steps
      print('[%d/%d][%d/%d] Loss_D: %.4f; Loss_G: %.4f' % (epoch, EPOCH, i+1, len(noise_loader), errD.item(), errG.item()))
      
        

In [ ]:
print(fake22.shape)